In [1]:
import requests
import pandas as pd
import numpy as np
import ast

- Cargamos ambos ficheros de ataques y clima

In [2]:
df_attacks = pd.read_csv("attacks_limpieza_completa.csv", index_col = 0)
df_attacks.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [3]:
df_attacks['country'].unique()

array(['usa', 'australia', 'mexico', 'brazil', 'england', 'south africa',
       'thailand', 'costa rica', 'maldives', 'bahamas', 'new caledonia',
       'ecuador', 'malaysia', 'libya', 'Unknown', 'cuba', 'mauritius',
       'new zealand', 'spain', 'samoa', 'solomon islands', 'japan',
       'egypt', 'st helena, british overseas territory', 'comoros',
       'reunion', 'french polynesia', 'united kingdom',
       'united arab emirates', 'philippines', 'indonesia', 'china',
       'columbia', 'cape verde', 'fiji', 'dominican republic',
       'cayman islands', 'aruba', 'mozambique', 'puerto rico', 'italy',
       'atlantic ocean', 'greece', 'st. martin', 'france',
       'papua new guinea', 'trinidad & tobago', 'kiribati', 'israel',
       'diego garcia', 'taiwan', 'jamaica', 'palestinian territories',
       'guam', 'seychelles', 'belize', 'nigeria', 'tonga', 'scotland',
       'canada', 'croatia', 'saudi arabia', 'chile', 'antigua', 'kenya',
       'russia', 'turks & caicos', 'united 

- Seleccionamos únicamente los ataques de los países pedidos

In [4]:
lista_paises = ['usa','australia','new zealand','south africa','papua new guinea']

In [7]:
df_paises = df_attacks[df_attacks['country'].isin(lista_paises)]
df_paises.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [6]:
df_paises.shape

(4335, 21)

- Cargamos el segundo csv co los datos del clima

In [8]:
df_clima = pd.read_csv("datos_clima.csv", index_col = 0)
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 200, 'speed':...",22,2,8,1018,none,0,0,225,2,usa
1,6,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 185, 'speed':...",22,2,6,1021,none,0,0,250,2,usa


In [10]:
df_clima.reset_index(inplace=True)

- Queremos hacer el groupby por para unir la tabla del clima con la ataques pero para ello, necesitamos limpiar un poco algunas de nuestras columnas.

In [11]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [12]:
df_clima.wind_profile[0]

[{'layer': '950mb', 'direction': 200, 'speed': 2},
 {'layer': '900mb', 'direction': 175, 'speed': 3},
 {'layer': '850mb', 'direction': 180, 'speed': 3},
 {'layer': '800mb', 'direction': 190, 'speed': 3},
 {'layer': '750mb', 'direction': 200, 'speed': 3},
 {'layer': '700mb', 'direction': 230, 'speed': 3},
 {'layer': '650mb', 'direction': 245, 'speed': 4},
 {'layer': '600mb', 'direction': 255, 'speed': 5},
 {'layer': '550mb', 'direction': 255, 'speed': 5},
 {'layer': '500mb', 'direction': 255, 'speed': 5},
 {'layer': '450mb', 'direction': 260, 'speed': 5},
 {'layer': '400mb', 'direction': 250, 'speed': 5},
 {'layer': '350mb', 'direction': 250, 'speed': 5},
 {'layer': '300mb', 'direction': 230, 'speed': 5},
 {'layer': '250mb', 'direction': 240, 'speed': 5},
 {'layer': '200mb', 'direction': 230, 'speed': 5}]

In [13]:
df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

In [14]:
df_clima["rh_profile"][0]

[{'layer': '950mb', 'rh': 4},
 {'layer': '900mb', 'rh': 3},
 {'layer': '850mb', 'rh': 2},
 {'layer': '800mb', 'rh': 0},
 {'layer': '750mb', 'rh': 2},
 {'layer': '700mb', 'rh': 4},
 {'layer': '650mb', 'rh': 5},
 {'layer': '600mb', 'rh': 6},
 {'layer': '550mb', 'rh': 7},
 {'layer': '500mb', 'rh': 3},
 {'layer': '450mb', 'rh': 5},
 {'layer': '400mb', 'rh': 11},
 {'layer': '350mb', 'rh': 15},
 {'layer': '300mb', 'rh': 10},
 {'layer': '250mb', 'rh': 8},
 {'layer': '200mb', 'rh': 9}]

In [15]:
wind = df_clima['wind_profile'].apply(pd.Series)
wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 200, 'speed': 2}","{'layer': '900mb', 'direction': 175, 'speed': 3}","{'layer': '850mb', 'direction': 180, 'speed': 3}","{'layer': '800mb', 'direction': 190, 'speed': 3}","{'layer': '750mb', 'direction': 200, 'speed': 3}","{'layer': '700mb', 'direction': 230, 'speed': 3}","{'layer': '650mb', 'direction': 245, 'speed': 4}","{'layer': '600mb', 'direction': 255, 'speed': 5}","{'layer': '550mb', 'direction': 255, 'speed': 5}","{'layer': '500mb', 'direction': 255, 'speed': 5}","{'layer': '450mb', 'direction': 260, 'speed': 5}","{'layer': '400mb', 'direction': 250, 'speed': 5}","{'layer': '350mb', 'direction': 250, 'speed': 5}","{'layer': '300mb', 'direction': 230, 'speed': 5}","{'layer': '250mb', 'direction': 240, 'speed': 5}","{'layer': '200mb', 'direction': 230, 'speed': 5}"
1,"{'layer': '950mb', 'direction': 185, 'speed': 2}","{'layer': '900mb', 'direction': 175, 'speed': 3}","{'layer': '850mb', 'direction': 175, 'speed': 4}","{'layer': '800mb', 'direction': 190, 'speed': 4}","{'layer': '750mb', 'direction': 200, 'speed': 3}","{'layer': '700mb', 'direction': 215, 'speed': 3}","{'layer': '650mb', 'direction': 250, 'speed': 3}","{'layer': '600mb', 'direction': 265, 'speed': 4}","{'layer': '550mb', 'direction': 265, 'speed': 5}","{'layer': '500mb', 'direction': 260, 'speed': 5}","{'layer': '450mb', 'direction': 260, 'speed': 4}","{'layer': '400mb', 'direction': 260, 'speed': 5}","{'layer': '350mb', 'direction': 260, 'speed': 5}","{'layer': '300mb', 'direction': 235, 'speed': 5}","{'layer': '250mb', 'direction': 245, 'speed': 5}","{'layer': '200mb', 'direction': 235, 'speed': 5}"


In [14]:
for i in range(len(wind.columns)):
    nombre = "wind_" + str(wind[i].apply(pd.Series)["layer"][0]) 
    valores = list(wind[i].apply(pd.Series)["speed"] )
    df_clima.insert(i, nombre, valores)

In [15]:
rh = df_clima['rh_profile'].apply(pd.Series) 
rh.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 4}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 0}","{'layer': '750mb', 'rh': 2}","{'layer': '700mb', 'rh': 4}","{'layer': '650mb', 'rh': 5}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 11}","{'layer': '350mb', 'rh': 15}","{'layer': '300mb', 'rh': 10}","{'layer': '250mb', 'rh': 8}","{'layer': '200mb', 'rh': 9}"
1,"{'layer': '950mb', 'rh': 5}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 0}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 4}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 2}","{'layer': '450mb', 'rh': 0}","{'layer': '400mb', 'rh': 11}","{'layer': '350mb', 'rh': 13}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 8}"
2,"{'layer': '950mb', 'rh': 6}","{'layer': '900mb', 'rh': 4}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 0}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 1}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 11}","{'layer': '300mb', 'rh': 1}","{'layer': '250mb', 'rh': 9}","{'layer': '200mb', 'rh': 10}"
3,"{'layer': '950mb', 'rh': 6}","{'layer': '900mb', 'rh': 4}","{'layer': '850mb', 'rh': 1}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 4}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 6}","{'layer': '400mb', 'rh': 10}","{'layer': '350mb', 'rh': 7}","{'layer': '300mb', 'rh': 3}","{'layer': '250mb', 'rh': 10}","{'layer': '200mb', 'rh': 14}"
4,"{'layer': '950mb', 'rh': 6}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 1}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 9}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 6}","{'layer': '400mb', 'rh': 12}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 11}"


In [16]:
for i in range(len(rh.columns)): 
    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    valores = list(rh[i].apply(pd.Series)["rh"]) 
    df_clima.insert(i, nombre, valores)

En este bucle creamos el nombre de las columnas (nombre) y sus valores (valores) uno a uno así : rh + el valor de la layer + el valor que tenga rh en esa iteración.

In [17]:
df_clima.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,4,3,2,0,2,4,5,6,7,3,...,22,2,8,1018,none,0,0,225,2,usa
1,5,3,2,0,3,4,4,7,11,2,...,22,2,6,1021,none,0,0,250,2,usa
2,6,4,2,0,1,3,3,6,11,1,...,23,2,4,1022,none,0,0,95,2,usa
3,6,4,1,-1,0,1,3,6,4,3,...,23,-1,4,1022,none,0,0,110,2,usa
4,6,3,1,-1,0,1,3,6,9,5,...,22,-1,5,1022,none,0,0,120,2,usa


- Comprobamos que se han desempacado en todas la columnas

In [18]:
df_clima.columns

Index(['rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb',
       'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb',
       'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'wind_950mb',
       'wind_900mb', 'wind_850mb', 'wind_800mb', 'wind_750mb', 'wind_700mb',
       'wind_650mb', 'wind_600mb', 'wind_550mb', 'wind_500mb', 'wind_450mb',
       'wind_400mb', 'wind_350mb', 'wind_300mb', 'wind_250mb', 'wind_200mb',
       'index', 'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'country'],
      dtype='object')

- Ya podemos hacer el groupby con las columnas limpias y relucientes!

In [19]:
df_group_clima = df_clima.groupby('country').mean().reset_index()
df_group_clima.head()

,country,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,...,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,australia,12.265625,11.687500,9.343750,8.015625,5.828125,3.984375,3.500000,3.390625,2.937500,...,-9999.0,-9999.0,23.156250,1.875000,10.078125,1017.968750,3.796875,0.0,148.437500,2.593750
1,new zealand,11.203125,12.750000,12.906250,12.468750,9.484375,6.703125,5.328125,4.765625,4.546875,...,-9999.0,-9999.0,11.562500,3.437500,10.500000,1003.031250,4.390625,0.0,255.312500,4.875000
2,papua new guinea,12.968750,11.796875,6.359375,2.781250,-0.125000,-1.093750,0.312500,0.593750,0.078125,...,-9999.0,-9999.0,26.593750,-0.640625,10.359375,1011.562500,1.843750,0.0,96.328125,3.203125
3,south africa,11.000000,13.156250,12.265625,6.578125,3.468750,1.156250,-0.140625,0.468750,1.062500,...,-9999.0,-9999.0,19.718750,3.968750,8.765625,1019.171875,5.859375,0.0,214.765625,3.562500
4,usa,5.875000,4.875000,4.546875,3.734375,3.500000,3.921875,3.437500,3.109375,3.984375,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.937500,0.0,139.531250,2.421875


In [20]:
df_union = pd.merge(df_attacks , df_group_clima , on='country', how = "inner")

In [21]:
df_union.head(6)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.9375,0.0,139.53125,2.421875
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.9375,0.0,139.53125,2.421875
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,-100.445882,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.9375,0.0,139.53125,2.421875
3,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,39.78373,-100.445882,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.9375,0.0,139.53125,2.421875
4,2018,Unprovoked,usa,15.0,Lemon shark,May,N,M,39.78373,-100.445882,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.9375,0.0,139.53125,2.421875
5,2018,Unprovoked,usa,12.0,Unespecific,May,N,M,39.78373,-100.445882,...,-9999.0,-9999.0,22.203125,1.671875,5.859375,1015.984375,0.9375,0.0,139.53125,2.421875


- Guardamos todo en un csv

In [22]:
df_union.to_csv('union.csv')

In [23]:
df_union.columns

Index(['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'latitud', 'longitud', 'country2', 'fatal_N', 'fatal_Unknown',
       'fatal_Y', 'fatal_N.1', 'fatal_Unknown.1', 'fatal_Y.1', 'species_.1',
       'fecha_limpia.1', 'type.1', 'age_NORM', 'rh_950mb', 'rh_900mb',
       'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb',
       'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb',
       'rh_250mb', 'rh_200mb', 'wind_950mb', 'wind_900mb', 'wind_850mb',
       'wind_800mb', 'wind_750mb', 'wind_700mb', 'wind_650mb', 'wind_600mb',
       'wind_550mb', 'wind_500mb', 'wind_450mb', 'wind_400mb', 'wind_350mb',
       'wind_300mb', 'wind_250mb', 'wind_200mb', 'index', 'timepoint',
       'cloudcover', 'highcloud', 'midcloud', 'lowcloud', 'temp2m',
       'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed'],
      dtype='object')

In [24]:
df_clima.to_csv("datos_clima_limpio.csv")